In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from tqdm.auto import tqdm  # progress bar
import tensorflow as tf
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Tokenize text & create dataset

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

def create_dataset(df, label_name = 'ArgumentLevel', label_number = 5, batch_size = 16, max_length = 256):

    # Tokenize text
    X_input_ids = np.zeros((len(df), max_length))
    X_attn_masks = np.zeros((len(df), max_length))

    for i, text in tqdm(enumerate(df['Content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = max_length,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
            return_tensors = 'tf'
        )
        
        X_input_ids[i,:] = tokenized_text.input_ids
        X_attn_masks[i,:] = tokenized_text.attention_mask
    

    # Generate labels - One-hot encoding
    labels = np.zeros((len(df), label_number)) 
    labels[np.arange(len(df)), df[label_name].values] = 1


    # Create dataset object
    dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

    # Map dataset
    dataset = dataset.map(DatasetMapFunction)

    # shuffle data
    dataset = dataset.shuffle(400,reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True) # drop remainder 179/16 = 11...3, drop 3 data

    return dataset

Split training and validation data 

In [ ]:
def split_dataset(dataset, p = 0.5, batch_size = 16):
    # p - training data size %
    train_size = int((len(df)//batch_size)*p)
    
    train_dataset = dataset.take(train_size) # take the first 8 batches
    val_dataset = dataset.skip(train_size) # take the last 3 batches

    return train_dataset, val_dataset

Create model

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(label_number = 5, max_length = 256):

    # input layer
    input_ids = tf.keras.layers.Input(shape = (max_length), name = 'input_ids', dtype = 'int32')
    # attention layer
    attention_masks = tf.keras.layers.Input(shape = (max_length), name = 'attention_mask', dtype = 'int32')

    # bert layer
    bert_layer = bert_model.bert(input_ids, attention_mask=attention_masks)[1] 

    # intermediate layer
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name = 'intermediate_layer')(bert_layer)

    # output layer - 5 layers because 5 label classes
    output_layer = tf.keras.layers.Dense(label_number, activation='softmax', name = 'output_layer')(intermediate_layer)

    # Create model object
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)



    # Add optimizer & Loss function & Accuracy metrics
    optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracry')

    model.compile(optimizer = optim, loss = loss_func, metrics = [acc])

    return model


========= 分界线 ==========
==========================

In [233]:
year = 'Y2'
label_name = 'ArgumentLevel'
label_number = 5

run_num = 5

In [234]:
folderpath = '/content/drive/MyDrive/nlp-physicseducation/outputs'
filepath = '/sections/labels_cleaned_{year}.csv'.format(year = year)

Load data

In [235]:
df = pd.read_csv(folderpath+filepath)
# df.head()
df['ArgumentLevel'] = df['ArgumentLevel'].replace({'superficial': 0, 'extended': 1, 'deep': 2, 'expert': 3, 'prediction': 4})
df['ReasoningLevel'] = df['ReasoningLevel'].replace({'bal': 0, 'the': 1, 'exp': 2, 'none': 3})
# df['ReasoningLevel'].value_counts()

In [236]:
for i in range(run_num):
    # about dataset
    dataset = create_dataset(df, label_name = label_name, label_number = label_number, batch_size = 16, max_length=512) # create dataset
    train_dataset, val_dataset = split_dataset(dataset, p = 0.8, batch_size = 16) # split training and validation data 


    # about model
    model = create_model(label_number = label_number, max_length = 512)   # Create model


    # about train and validate
    history = model.fit(     # Train & Validate model (Fine-tuning)
        train_dataset,
        validation_data = val_dataset,
        epochs = 4
    )


    history_saveto = folderpath + '/BERT20230217/{year}/{label_name}/{time}_{year}_{label_name}_{size}trainsize_4epochs_16batchsize_1e-5lr.npy'.format(
        time = datetime.now().strftime("%H%M%S"), 
        year = year, 
        label_name = label_name, 
        size = 8
        )
    np.save(history_saveto, history.history) # save history


    # clear model after training
    tf.keras.backend.clear_session()
    import gc
    gc.collect()
    del model

0it [00:00, ?it/s]

Epoch 1/4
4/4 [==============================] - 39s 888ms/step - loss: 1.8808 - accuracry: 0.2344 - val_loss: 1.6624 - val_accuracry: 0.1250
Epoch 2/4
4/4 [==============================] - 1s 268ms/step - loss: 1.6597 - accuracry: 0.2188 - val_loss: 1.6679 - val_accuracry: 0.1250
Epoch 3/4
4/4 [==============================] - 1s 267ms/step - loss: 1.5433 - accuracry: 0.2500 - val_loss: 1.3857 - val_accuracry: 0.3750
Epoch 4/4
4/4 [==============================] - 1s 265ms/step - loss: 1.3702 - accuracry: 0.3906 - val_loss: 1.2929 - val_accuracry: 0.4375


0it [00:00, ?it/s]

Epoch 1/4
4/4 [==============================] - 38s 854ms/step - loss: 1.6465 - accuracry: 0.2188 - val_loss: 1.2671 - val_accuracry: 0.6875
Epoch 2/4
4/4 [==============================] - 1s 269ms/step - loss: 1.4486 - accuracry: 0.4062 - val_loss: 1.2466 - val_accuracry: 0.5625
Epoch 3/4
4/4 [==============================] - 1s 268ms/step - loss: 1.2941 - accuracry: 0.5000 - val_loss: 1.3201 - val_accuracry: 0.6875
Epoch 4/4
4/4 [==============================] - 1s 272ms/step - loss: 1.1916 - accuracry: 0.5938 - val_loss: 0.9461 - val_accuracry: 0.6875


0it [00:00, ?it/s]

Epoch 1/4
4/4 [==============================] - 38s 866ms/step - loss: 1.8013 - accuracry: 0.0781 - val_loss: 1.6396 - val_accuracry: 0.0625
Epoch 2/4
4/4 [==============================] - 1s 267ms/step - loss: 1.5156 - accuracry: 0.2969 - val_loss: 1.3440 - val_accuracry: 0.7500
Epoch 3/4
4/4 [==============================] - 1s 267ms/step - loss: 1.3483 - accuracry: 0.5469 - val_loss: 1.1764 - val_accuracry: 0.7500
Epoch 4/4
4/4 [==============================] - 1s 267ms/step - loss: 1.2263 - accuracry: 0.6094 - val_loss: 1.0970 - val_accuracry: 0.5625


0it [00:00, ?it/s]

Epoch 1/4
4/4 [==============================] - 39s 884ms/step - loss: 1.6701 - accuracry: 0.2812 - val_loss: 1.3264 - val_accuracry: 0.5000
Epoch 2/4
4/4 [==============================] - 1s 269ms/step - loss: 1.3851 - accuracry: 0.4219 - val_loss: 1.1469 - val_accuracry: 0.8125
Epoch 3/4
4/4 [==============================] - 1s 266ms/step - loss: 1.2323 - accuracry: 0.5625 - val_loss: 1.0786 - val_accuracry: 0.6875
Epoch 4/4
4/4 [==============================] - 1s 268ms/step - loss: 1.1029 - accuracry: 0.6406 - val_loss: 1.1879 - val_accuracry: 0.5625


0it [00:00, ?it/s]

Epoch 1/4
4/4 [==============================] - 39s 862ms/step - loss: 1.6353 - accuracry: 0.3594 - val_loss: 1.4111 - val_accuracry: 0.5000
Epoch 2/4
4/4 [==============================] - 1s 271ms/step - loss: 1.4716 - accuracry: 0.4062 - val_loss: 1.1297 - val_accuracry: 0.7500
Epoch 3/4
4/4 [==============================] - 1s 267ms/step - loss: 1.3366 - accuracry: 0.5469 - val_loss: 1.1090 - val_accuracry: 0.6875
Epoch 4/4
4/4 [==============================] - 1s 286ms/step - loss: 1.1802 - accuracry: 0.6094 - val_loss: 1.1268 - val_accuracry: 0.5000


Save model

In [168]:
model_saveto = './}_{}_trainsize{}'
model.save('Y1Y2_argument_model')

NameError: ignored